# Tokenizer


In [1]:
ord("h") # gives unicode code point of the character; not string

104

In [2]:
ord("👋")

128075

In [3]:
"안녕하세요 👋 (hello in Korean)"

'안녕하세요 👋 (hello in Korean)'

In [4]:
[ord(x) for x in "안녕하세요 👋 (hello in Korean)"]

[50504,
 45397,
 54616,
 49464,
 50836,
 32,
 128075,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 41]

Why can't we use unicode points directly as tokens? 

One reason is, in that case vocabulary size will increase drastically. Unicode standard is alive and it is continuously changing. So it is not stable. We need something better.

That better is encodings. Unicode standard defined 3 types of encoding UTF-8, 16 and 32

UTF-8 is most popular. It takes unicode points and translate them into bytes streams and bytes are from 1 to 4 bytes long. Variable length encoding.

In [5]:
"안녕하세요 👋 (hello in Korean)".encode("utf-8") # gives byte stream

b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94 \xf0\x9f\x91\x8b (hello in Korean)'

In [6]:
# for raw bytes
list("안녕하세요 👋 (hello in Korean)".encode("utf-8"))

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 41]

BPE Implmentation

In [7]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."

tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to list of integers in range 0..255 for convenience

print("----")
print(text)
print("----")
print("length: ", len(text))
print("----")
print(tokens)
print("length: ", len(tokens))

----
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
----
length:  533
----
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 

now we will find the most frequent pair of characters in the text and merge them recursively.

In [8]:
def get_stats(ids): # expects a list of integers
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
        # print(pair)
    return counts

stats = get_stats(tokens)
# print(stats)
print(sorted(((v, k) for k, v in stats.items()), reverse=True))

[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (

In [9]:
chr(101), chr(32)

('e', ' ')

In [10]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [11]:
def merge(ids, pair, idx):
    # in the list of int(ids), replace all consecutive occurrences of pair with the new token idx
    new_ids = []
    i = 0
    while i < len(ids):
        # if we are not at the very last position AND their pair matches, replace it
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i+= 1
            
    return new_ids

print(merge([5,6,6,7,9,1], (6,7), 99)) # toy example

[5, 6, 99, 9, 1]


In [12]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length: ", len(tokens2))

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111, 32, 226, 128, 156

now we will again find the most frequent pair of characters in the new text and merge them. This we repeat for n times. "n" is hyperparameter. Usually there exist sweet spot for n. The more we iterate we larger the vocabulary size we will get and shorter the tokens will be.

In [13]:
# read text from file
with open("unicode.txt", "r") as f:
    text = f.read()
    tokens = text.encode("utf-8")
    tokens = list(map(int, tokens))

In [14]:
vocab_size = 276 # the desired final vocabulary size
num_merges = vocab_size - 256 # we start with 256 tokens
ids = list(tokens) # copy so that we don't destroy the original

merges = {} # (int, int) -> int

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merging {pair} into new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

merging (101, 32) into new token 256
merging (105, 110) into new token 257
merging (115, 32) into new token 258
merging (116, 104) into new token 259
merging (101, 114) into new token 260
merging (99, 111) into new token 261
merging (116, 32) into new token 262
merging (226, 128) into new token 263
merging (44, 32) into new token 264
merging (97, 110) into new token 265
merging (111, 114) into new token 266
merging (100, 32) into new token 267
merging (97, 114) into new token 268
merging (101, 110) into new token 269
merging (257, 103) into new token 270
merging (261, 100) into new token 271
merging (121, 32) into new token 272
merging (46, 32) into new token 273
merging (97, 108) into new token 274
merging (259, 256) into new token 275


In [15]:
print("tokens length: ", len(tokens))
print("ids length: ", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length:  24636
ids length:  19484
compression ratio: 1.26X


tokenizier is completely seperate object from LLM. It has its own training set (which could be different from that LLM set). It is preprocessing stage 

decoding:

given a sequence of integers in the range [0, vocab_size] what is text?

In [16]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
    

def decode(ids):
    # given ids (list of integers), return Python String
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace") # we need to replace errors because we may have unknown tokens. For eg: 128, will not fit the UTF 8 standard. So it will throw error. We need to replace it with something else. Python provides a way to replace it with a question mark.
    return text

decode([128])

'�'

encoding

The other way round: Given string, what are the tokens?

In [17]:
def encode(text):
    tokens = list(text.encode("utf-8")) # list of integers of those raw bytes
    while True:
        stats = get_stats(tokens)
        pair = min(stats, key = lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("hello world!")) # this is not actual implementation yet, because if we pass single character then "stats" var will be empty and thus "min" can be calculated and will give error

[104, 101, 108, 108, 111, 32, 119, 266, 108, 100, 33]


Fix:

In [18]:
def encode(text):
    tokens = list(text.encode("utf-8")) # list of integers of those raw bytes
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key = lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("h"))

[104]


few test cases

In [19]:
decode(encode("hello world!"))

'hello world!'

In [20]:
text2 = decode(encode(text))
print(text2 == text)

True


In [21]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


Forced splits using regex patterns

In [30]:
import regex as re

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello World"))
print(re.findall(gpt2pat, "Hello World how are you")) # ?\p{L} seperates out words
print(re.findall(gpt2pat, "Hello World123 how are you")) # ?\p{N} seperates out numbers
print(re.findall(gpt2pat, "Hello've World123 how are you")) # matching apostrophe words
# ^\s\p{L}\p{N}]+ is to match any character that is not a space, letter or number basically a punctuation
print(re.findall(gpt2pat, "Hello've World123 how are you!!!?"))
print(re.findall(gpt2pat, "Hello've World123 how are               you!!!?")) # \s+(?!\S) is to match multiple spaces
print(re.findall(gpt2pat, "Hello've World123 how are     you!!!?        ")) # \s+ is to capture trailing spaces

['Hello', ' World']
['Hello', ' World', ' how', ' are', ' you']
['Hello', ' World', '123', ' how', ' are', ' you']
['Hello', "'ve", ' World', '123', ' how', ' are', ' you']
['Hello', "'ve", ' World', '123', ' how', ' are', ' you', '!!!?']
['Hello', "'ve", ' World', '123', ' how', ' are', '              ', ' you', '!!!?']
['Hello', "'ve", ' World', '123', ' how', ' are', '    ', ' you', '!!!?', '        ']


In [32]:
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("       Hello World!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("       Hello World!!!"))

[220, 220, 220, 220, 220, 220, 18435, 2159, 10185]
[996, 22691, 4435, 12340]


special tokens

complete the exercise, and try to build via own from that exercise. MBPE

sentencepiece